In [2]:
from math import log2
x = [3, 1, 7, 0, 4, 1, 6, 3]
y = [0 for _ in x]
n = len(x)
print(int(log2(n)))

3


In [5]:
def scan1(x, n):
    xy = [_ for _ in x]
    print(xy)
    logn = int(log2(n))
    for d in range(logn):
        print('d: ', d)
        for i in range(0, n):
            step = 2**(d+1)
            if (i % step ==0):
                ai = i + 2**d -1
                bi = i + 2**(d+1) - 1 
                xy[bi] = xy[ai] + xy[bi]
            
    return xy

s = scan1(x, n)
print(s)

[3, 1, 7, 0, 4, 1, 6, 3]
d:  0
d:  1
d:  2
[3, 4, 7, 11, 4, 5, 6, 25]


In [6]:
d = [0, 1, 2, 3, 4]
two_power_d = [1 << x for x in d]
print(two_power_d)


[1, 2, 4, 8, 16]
